In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 50.6 MB/s 
     |████████████████████████████████| 596 kB 56.5 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install aitextgen

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 572 kB 3.4 MB/s 
     |████████████████████████████████| 87 kB 8.7 MB/s 
     |████████████████████████████████| 584 kB 51.0 MB/s 
     |████████████████████████████████| 409 kB 41.4 MB/s 
     |████████████████████████████████| 140 kB 64.2 MB/s 
     |████████████████████████████████| 1.1 MB 48.2 MB/s 
     |████████████████████████████████| 271 kB 65.8 MB/s 
     |████████████████████████████████| 94 kB 4.8 MB/s 
     |████████████████████████████████| 144 kB 63.4 MB/s 
  Created wheel for aitextgen: filename=aitextgen-0.5.2-py3-none-any.whl size=575905 sha256=124e682f17712247232d4ee2bcba0f9385504ef0a76fb97801b92064f0d0432d
  Stored in directory: /root/.cache/pip/wheels/83/e2/74/46c887b0989a51a7acee0c09551a3ae9d34b939fb4bea404a0
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=f6ca09c755b2756d7ed81cbfe40d7163c1

In [3]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/SMU_MITB_NLP/NLP project"

Mounted at /content/drive
/content/drive/MyDrive/SMU_MITB_NLP/NLP project


In [5]:
from aitextgen import aitextgen
from aitextgen.TokenDataset import TokenDataset, merge_datasets
from pytorch_lightning.loggers import TensorBoardLogger

In [6]:
import os
import torch
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np

In [7]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16962107030676088009
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 13266321408
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17685878727494914573
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [8]:
torch.cuda.is_available()

True

In [16]:
hf_model = "gpt2"
ai = aitextgen(model=hf_model, verbose=True)
ai.to_gpu()
out_dir = "./output/"

loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at aitextgen/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
   

In [17]:
path = "./songs_edit_revised"
songdata = []
for _, _, filenames in os.walk(path):
    for file in filenames:
        f = open(path + "/" + file, "r")
        song = f.read()
        songdata.append(song)
        songdata.append("\n<EOS>\n")
        #data = TokenDataset(path + "/" + file, line_by_line=True, block_size=32)
        f.close()

In [18]:
f = open("./sgsong_transformer.txt", "w")
for song in songdata:
    f.write(song)
f.close()

In [19]:
traindata = TokenDataset("./sgsong_transformer.txt", block_size=32)
#data_merged = merge_datasets(songdata)

  0%|          | 0/2469 [00:00<?, ?it/s]

In [20]:
ai.train(
    traindata,
    n_gpu = 1,
    num_steps = 5000,
    generate_every = 100,
    output_dir = out_dir,
    loggers = [TensorBoardLogger(out_dir)],
    freeze_layers = True,
    num_layers_freeze = 9,
    line_by_line = False,
    header = False,
)

pytorch_model.bin already exists in /./output/ and will be overwritten!
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is depre

  0%|          | 0/5000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2265: LightningDeprecationWarning: `trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7. Use `ProgressBarBase.get_metrics` instead.
  "`trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7."


100 steps reached: generating sample texts.
A new day awaits
The city of a thousand dreams awaits

<EOS>
<VERSEOS
A brand fresh day awaits
We are you you and me
We will always be
A brand new right
We will always be
We will be our dreams come, we will celebrate it
A brand new day awaits awaits
We will stand
<VERSE
A brand new day awaits
Singing this moment
And a brand new day awaits awaits
As we will share
What a brand new day awaits
A brand new day awaits
A brand new day awaits
Let us celebrate this moment
A brand new day awaits
A brand new day awaits
A brand new day awaits
A brand new day awaits
A brand new day awaits
A brand new day awaits awaits
A brand new day awaits
My dreams await is still moments
Let us celebrate this moment
A brand new day awaits
A brand new day awaits
A brand new day awaits
A brand new day awaits awaits
A brand new day awaits
A brand new day awaits
A brand new day awaits
A brand new day awaits
A brand new day awaits awaits
A brand new day awaits
A brand new da

Configuration saved in ./output/config.json
Model weights saved in ./output/pytorch_model.bin


...

Configuration saved in ./output/config.json


5,000 steps reached: saving model to /./output/


Model weights saved in ./output/pytorch_model.bin


5,000 steps reached: generating sample texts.
<VERSE>
Let us be together
aspire to achieve
No matter when,
No matter when,
Working hand in hand
Let us never be together, 
together hand in hand in hand
Let us always be
And we'll be
Together hand in joy and harmony



<VERSE>
Let us continue together, 
hands in hand in hand in hand
Let us
Let us grow this land
Let us learn to all our part of building something new strength
Let us
Let us all our destiny
Let us all we'll learn to all we all we do what we can be done
Let us Singapore
Let us do
Let us do
Let us to do
Let us
Let us all do
Let us do
Let us
Let us all we do
Let us
Let us all we can do
Let us
Let us
Let us
Let us share
Let's all we can do
Let us
Let us
Let us
Let us
Let us
Let's do
Let us
Let us all we can do
Let us all we do
together, 
Let us
Let us
Let us
Let us Singapore, 


Configuration saved in ./output/config.json
Model weights saved in ./output/pytorch_model.bin


In [21]:
prompt = input("Prompt:\n\n")
output = ai.generate(
    prompt = prompt,
    n = 6,
    min_len = None,
    max_len = 256,
    temperature = 0.8,
    do_sample = True,
    use_cache = True,
    early_stopping = False,
    num_beams = 1,
    top_k = 50,
    top_p = 0.75,
    repetition_penalty = 1.2,
    length_penalty = 1.0,
    no_repeat_ngram_size = 0,
    num_beam_groups = 1,
    diversity_penalty = 0.0,
    remove_invalid_values = True,
    return_as_list = True,
    lstrip = False,
    skip_special_tokens = False
)

for i, text in enumerate(output):
  print("\n============================\n")
  print(f"Song: {i}\n\n")
  print(text)

Prompt:

Multi-racial society


Song: 0


Multi-racial society
Sing a thousand different voices Sing in harmony
We will stand together as one united people, undivided ’s hand in hand
For Singapore, my homeland, this is where I belong
I know we're gonna be forevermore
All of us unified, 

To grow the fruits of unity
As one united people
Differentity
You'll always stay. For Singapore, my homeland
And all our strength
Together we do
It's here
Because of who you and me
What a million more than what we are
With each race
Let's here
In your skill to treasure
One destiny
So there is an open the way
Do whatever - mine
There's here
Like that path
Then unite
<EOS>
Maybe it
Walking tall
Is yours and everything out
Also let's here
When we'll learn something else
No matter when we'll find
Everydaydream
Whatever grows stronger by yourself and forge
This is worth
Here's here
Seeing Singapore, go
Shining tomorrow
That's here
Look forward
Have them
If we'll keep coming
Or see
Especially for Singapore
R